# Использование фреймворка для решения задач RL

Рассмотрим работу одного из фреймворков для решения задач RL: `stable_baselines3` (далее SB3).
Для установки можно использовать команду в консоли

    pip install stable-baselines3[extra]

Документация по `stable_baselines3`:

    https://stable-baselines3.readthedocs.io/en/master/

Обучение ИНС в этом фреймворке происходит с помощью библиотеки `PyTorch`.

In [3]:
import stable_baselines3
stable_baselines3.__version__

'2.5.0'

In [4]:
import gymnasium as gym
gym.__version__

'1.0.0'

Ключевым объектом для работы с этой библиотекой является экземпляр класса `BaseAlgorithm`. Он содержит в себе стратегию агента в виде ИНС и целый ряд параметров. Конкретные алгоритмы являются наследниками этого класса. Список алгоритмов можно найти по ссылке:

https://stable-baselines3.readthedocs.io/en/master/guide/algos.html

Рассмотрим пример на основе алгоритма PPO и среды `CartPole`.

In [7]:
# подгрузка алгоритма
from stable_baselines3 import PPO
# создание среды
env = gym.make("CartPole-v1")
# создание экземпляра алгоритма
model = PPO('MlpPolicy', env, verbose = 1) 

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


При создании экземпляра класса выбранного алгоритма требуется указать вид ИНС для стратегии. В данном случае это обычная полносвязная ИНС 'MlpPolicy' с параметрами по умолчанию. В дальнейших примерах будет показано, как можно задавать свою ИНС для алгоритма. 

Также требуется передать среду. Параметр `verbose = 1` означает, что при обучении на печать будет выводиться промежуточная информация.

При создании экземпляра алгоритма среда оборачивается в классы обёртки:
    `Monitor` - для слежения за выполнением,
    `DummyVecEnv` - обёртка с фейк-векторизацией.

Суть в том, что SB3 позволяет использовать агента одновременно в нескольких экземплярах среды. Это позволяет добавить скорости к обучению. При этом используется обёртка, которая векторизует эту среду. Далее, будет пример с векторизованной средой, однако, пока используем одну среду для обучения. И здесь используется обёртка по умолчанию `DummyVecEnv`.

Полный набор входных параметров для алгоритма можно вывести следующей командой.

In [29]:
PPO??

Init signature:
PPO(
    policy: Union[str, type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gymnasium.core.Env, ForwardRef('VecEnv'), str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    normalize_advantage: bool = True,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    rollout_buffer_class: Optional[type[stable_baselines3.common.buffers.RolloutBuffer]] = None,
    rollout_buffer_kwargs: Optional[dict[str, Any]] = None,
    target_kl: Optional[float] = None,
    stats_window_size: int = 100,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[dict[str, Any]]

Обучим модель методом `learn()`. Параметром можно указать число шагов в среде, использующихся для обучения.

In [34]:
model.learn(total_timesteps=25000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 21.5     |
|    ep_rew_mean     | 21.5     |
| time/              |          |
|    fps             | 1672     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 27.7        |
|    ep_rew_mean          | 27.7        |
| time/                   |             |
|    fps                  | 1225        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008297037 |
|    clip_fraction        | 0.0939      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00367    |
|    learning_rate        | 0.

Можно не выводить промежуточную информацию при обучении, для этого можно указать `verbose = 0` (или ничего не указывать, параметр `verbose` равен 0 по умолчанию). Однако, для отслеживания прогресса можно в методе `learn()` указать параметр `progress_bar=True`.

In [128]:
model = PPO('MlpPolicy', env, verbose = 0) 
model.learn(total_timesteps=25000, progress_bar=True)

Output()

Для демонстрации работы агента можно запустить симуляцию. Для получения оптимального действия у обученной модели надо вызвать метод `predict()`. Для методов типа PPO, где стратегия стохастична по определению, при вызове метода `predict()` рекомендуется указывать параметр `deterministic=True`.

In [133]:
env_eval = gym.make("CartPole-v1", render_mode = "human")
env_eval = gym.wrappers.RecordEpisodeStatistics(env_eval)

state, _ = env_eval.reset()

while True:
    action, _ = model.predict(state, deterministic=True)
    state, reward, terminated, truncated, _ = env_eval.step(action)
    if terminated or truncated:
        break
    
print("Доход:", env_eval.return_queue[-1])

env.close()

Доход: 500.0


Сохранить или загрузить модель можно командами 

    model.save('filename')
    model = PPO.load('filename', env=env)

In [137]:
import os
save_path = os.path.join('savedModels', 'SB3_PPO_CartPole')
model.save(save_path)

Для оценки полученной стратегии на большом числе эпизодов можно использовать также встроенный метод `evaluate_policy()`, указав для него необходимые параметры. 

In [149]:
from stable_baselines3.common.evaluation import evaluate_policy

model = PPO.load(save_path, env=env)
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)

print("Средний доход: ", mean_reward)
print("Стандартное отклонение: ", std_reward)
env.close()

Средний доход:  499.87
Стандартное отклонение:  1.293483668238606


Этот же метод можно использовать и для демонстрации работы агента.

In [147]:
evaluate_policy(model, env, n_eval_episodes=1, render=True)
env.close()

### Пример обучения с наблюдением процесса обучения через Tensorboard

Рассмотрим среду 'BipedalWalker-v3'. 

    https://gymnasium.farama.org/environments/box2d/bipedal_walker/

Это простая модель 4-суставного шагающего робота. Есть две версии:

     Обычная, со слегка неровной местностью,
     Сложная, с препятствиями.

Цель: набрать 300 очков за 1600 временных шагов.

In [9]:
env = gym.make('BipedalWalker-v3')
print(env.action_space)
print(env.observation_space)

Box(-1.0, 1.0, (4,), float32)
Box([-3.1415927 -5.        -5.        -5.        -3.1415927 -5.
 -3.1415927 -5.        -0.        -3.1415927 -5.        -3.1415927
 -5.        -0.        -1.        -1.        -1.        -1.
 -1.        -1.        -1.        -1.        -1.        -1.       ], [3.1415927 5.        5.        5.        3.1415927 5.        3.1415927
 5.        5.        3.1415927 5.        3.1415927 5.        5.
 1.        1.        1.        1.        1.        1.        1.
 1.        1.        1.       ], (24,), float32)


Запустим симуляцию со случайными действиями.

In [62]:
env = gym.make('BipedalWalker-v3', render_mode = 'human')
env = gym.wrappers.RecordEpisodeStatistics(env)

state, _ = env.reset()

while True:
    action = env.action_space.sample()
    state, reward, terminated, truncated, _ = env.step(action)
    if terminated or truncated:
        break
    
print("Доход:", env.return_queue[-1])
env.close()

Доход: -104.01821125072924


Это среда требует большего времени для обучения. За обучением удобно следить, например, с помощью методом библиотеки `Tensorboard`.

    pip install tensorboard

При создании модели в SB3 можно указать папку для создания tensorboard-логов.

In [23]:
env = gym.make('BipedalWalker-v3')

import os
logs_path = os.path.join('logs', 'ppo_bipedal_tensorboard')
model = PPO("MlpPolicy", env, tensorboard_log=logs_path)

Для запуска `tensorboard` надо в консоли в процессе обучения прописать специальную строчку, указав пусть к папке с логами.

Сначала построим путь к папке с логами.

In [37]:
os.getcwd() # получить путь к текущей папке, где лежит Python-ноутбук
full_logs_path = os.path.join(os.getcwd() ,'logs', 'ppo_bipedal_tensorboard')
print("".join(("tensorboard --logdir=",'"',full_logs_path,'"')))

tensorboard --logdir="C:\Users\AlexK\Documents\Python Scripts\RL\logs\ppo_bipedal_tensorboard"


Эту напечатанную строчку надо запустить в консоли после запуска обучения и перейти по ссылке которая там возникнет ( http://localhost:6006/)

Запустим обучение.

In [74]:
model.learn(total_timesteps=2_000_000, progress_bar=True)

Logging to logs\ppo_bipedal_tensorboard\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 375      |
|    ep_rew_mean     | -109     |
| time/              |          |
|    fps             | 1220     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 411         |
|    ep_rew_mean          | -106        |
| time/                   |             |
|    fps                  | 948         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007688776 |
|    clip_fraction        | 0.0783      |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.68       |
|    explained_variance   

In [81]:
save_path =  os.path.join('savedModels', 'SB3_PPO_BiPedal')
model.save(save_path)
print(save_path)

savedModels\SB3_PPO_BiPedal


In [45]:
from stable_baselines3.common.evaluation import evaluate_policy

save_path='savedModels/SB3_PPO_BiPedal'

env = gym.make('BipedalWalker-v3')
model = PPO.load(save_path, env=env)

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, render=False)
print("Средний доход: ", mean_reward)
print("Стандартное отклонение: ", std_reward)
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Средний доход:  308.3458813409088
Стандартное отклонение:  0.7299469793743381


In [47]:
env = gym.make('BipedalWalker-v3', render_mode = 'human')
evaluate_policy(model, env, n_eval_episodes=1, render=True)

(309.95424094819464, 0.0)

Добраться до используемой ИНС внутри модели можно, обратившись к полю `model.policy`. Архитектуру ИНС можно отобразить с помощью библиотеки `torchsummary`.

In [50]:
from torchsummary import summary

summary(model.policy, input_size=(1,24,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                   [-1, 24]               0
  FlattenExtractor-2                   [-1, 24]               0
            Linear-3                   [-1, 64]           1,600
              Tanh-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              Tanh-6                   [-1, 64]               0
            Linear-7                   [-1, 64]           1,600
              Tanh-8                   [-1, 64]               0
            Linear-9                   [-1, 64]           4,160
             Tanh-10                   [-1, 64]               0
     MlpExtractor-11       [[-1, 64], [-1, 64]]               0
           Linear-12                    [-1, 1]              65
           Linear-13                    [-1, 4]             260
Total params: 11,845
Trainable params: 

### Обратная связь при обучении (Callbacks)

В ходе обучения наибоее важный параметр, который надо отслеживать - 
это средний доход за эпизод. При достижении определённой величины дохода можно остановить обучение. 
Для того, чтобы настроить отслеживание среднего дохода за эпизод и прекращение обучения в библиотеке `SB3` есть специальные методы: `callback functions`.

С их помощью можно также можно сохранять модель по ходу обучения при выполнении некоторых условий и тд.

Создадим два callback-метода:

    1. Остановить обучение по достижении цели
    2. Сохранить модель по достижении нового лучшего результата

Для примера рассмотрим среду `CartPole-v1`.

In [54]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_PPO_CartPole')
save_path_best = os.path.join('savedModels', 'SB3_PPO_CartPole_best')
log_path = os.path.join('logs', 'SB3_PPO_CartPole')

env = gym.make("CartPole-v1")

In [56]:
# создание callback-методов
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=400, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=5000, 
                             best_model_save_path=save_path_best, 
                             verbose=1)

То есть каждые 5000 временных шагов будет вызываться EvalCallback() для проверки качества текущей модели. Если модель стала лучше, то вызывается `StopTrainingOnRewardThreshold()` для проверки того, стало ли качество модели выше, чем целевое значение. 

In [59]:
model = PPO('MlpPolicy', env, tensorboard_log=log_path)
model.learn(total_timesteps=20000, callback=eval_callback, progress_bar=True)

Output()

Eval num_timesteps=5000, episode_reward=339.20 +/- 165.53

Episode length: 339.20 +/- 165.53

New best mean reward!

Eval num_timesteps=10000, episode_reward=251.80 +/- 83.61

Episode length: 251.80 +/- 83.61

Eval num_timesteps=15000, episode_reward=405.80 +/- 79.16

Episode length: 405.80 +/- 79.16

New best mean reward!

Stopping training because the mean reward 405.80  is above the threshold 400

In [18]:
model.save(save_path)

In [22]:
load_path_best = os.path.join(save_path_best,"best_model")
load_path_best

'savedModels\\SB3_PPO_CartPole_best\\best_model'

In [26]:
model = PPO.load(load_path_best, env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [32]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)

print("Средний доход: ", mean_reward)
print("Стандартное отклонение: ", std_reward)
env.close()

Средний доход:  356.74
Стандартное отклонение:  164.91171092436096


### Пример

Рассмотрим среду "Pendulum-v1" и применим для обучения агента метод PPO.

https://www.gymlibrary.dev/environments/classic_control/pendulum/

In [61]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [63]:
env = gym.make("Pendulum-v1", render_mode = 'human')
env = gym.wrappers.RecordEpisodeStatistics(env)

state, _ = env.reset()

while True:
    action = env.action_space.sample()
    state, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        break
    
print("Доход:", env.return_queue[-1])

env.close()

Доход: -1029.790284745965


In [65]:
import os
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_PPO_Pendulum')
save_path_best = os.path.join('savedModels', 'SB3_PPO_Pendulum_best')
log_path = os.path.join('logs', 'SB3_PPO_Pendulum')

env = gym.make("Pendulum-v1")

# создание callback-методов
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=-170, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path_best, 
                             verbose=1)

model = PPO('MlpPolicy', env, tensorboard_log=log_path)

In [67]:
from torchsummary import summary

summary(model.policy, input_size=(1,3,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                    [-1, 3]               0
  FlattenExtractor-2                    [-1, 3]               0
            Linear-3                   [-1, 64]             256
              Tanh-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              Tanh-6                   [-1, 64]               0
            Linear-7                   [-1, 64]             256
              Tanh-8                   [-1, 64]               0
            Linear-9                   [-1, 64]           4,160
             Tanh-10                   [-1, 64]               0
     MlpExtractor-11       [[-1, 64], [-1, 64]]               0
           Linear-12                    [-1, 1]              65
           Linear-13                    [-1, 1]              65
Total params: 8,962
Trainable params: 8

Путь к логам Tensorboard.

In [72]:
full_logs_path = os.path.join(os.getcwd() ,log_path)
print("".join(("tensorboard --logdir=",'"',full_logs_path,'"')))

tensorboard --logdir="C:\Users\AlexK\Documents\Python Scripts\RL\logs\SB3_PPO_Pendulum"


In [69]:
model.learn(total_timesteps=200000, callback=eval_callback, progress_bar=True)

Output()

Eval num_timesteps=10000, episode_reward=-1062.03 +/- 208.52

Episode length: 200.00 +/- 0.00

New best mean reward!

Eval num_timesteps=20000, episode_reward=-1117.18 +/- 297.28

...

Eval num_timesteps=190000, episode_reward=-888.51 +/- 77.62

Episode length: 200.00 +/- 0.00

Eval num_timesteps=200000, episode_reward=-835.36 +/- 140.44

Episode length: 200.00 +/- 0.00

New best mean reward!

In [75]:
model.save(save_path)

In [77]:
load_path_best = os.path.join(save_path_best,"best_model")
env = gym.make("Pendulum-v1")
model = PPO.load(load_path_best, env=env)

In [79]:
env = gym.make("Pendulum-v1", render_mode = 'human')
gain, _ = evaluate_policy(model, env, n_eval_episodes=1, render=True)
print(gain)
env.close()

-934.7934630326927


### Смена дефолтной архитектуры сети

Архитертура ИНС, которая по умолчанию задаётся при вызове "MlpPolicy" у алгоритма обучения, может быть изменена.

Для метода PPO начальные слои ИНС отвечают за извлечение признаков из состояний на входе, эти слои общие для актора и критика. После этих слоев сеть делится на две части: одна для актора, другая для критика. 

Для простых по структуре состояний (например, в виде векторов) слои с извлечением признаков пусты. Для смены ахритектуры дальнейших слоёв можно использовать синтаксис

    policy_kwargs = dict(net_arch=dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128]))
    model = PPO('MlpPolicy', env, policy_kwargs=policy_kwargs)

То есть для каждой ветки ИНС будет создано по 4 слоя по 128 нейронов. См. подробности

https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html

Применим рассмотренные методы для более сложной задачи. 

### Пример со сменой архитектуры ИНС

Рассмотрим среду "InvertedDoublePendulum-v5".

In [81]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [83]:
env = gym.make("InvertedDoublePendulum-v5", render_mode = 'human')
env = gym.wrappers.RecordEpisodeStatistics(env)

state, _ = env.reset()

while True:
    action = env.action_space.sample()
    state, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        break
    
print("Доход:", env.return_queue[-1])

env.close()

Доход: 44.23927123895431


In [85]:
import os
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_PPO_InvPendulum')
save_path_best = os.path.join('savedModels', 'SB3_PPO_InvPendulum_best')
log_path = os.path.join('logs', 'SB3_PPO_InvPendulum')

env = gym.make("InvertedDoublePendulum-v5")

# создание callback
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=3000, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path_best, 
                             verbose=1)

#model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

policy_kwargs = dict(net_arch = dict(pi=[96, 96, 96], vf=[96, 96, 96]))
model = PPO('MlpPolicy', env, policy_kwargs=policy_kwargs, tensorboard_log=log_path)

In [87]:
summary(model.policy, input_size=(1,9,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                    [-1, 9]               0
  FlattenExtractor-2                    [-1, 9]               0
            Linear-3                   [-1, 96]             960
              Tanh-4                   [-1, 96]               0
            Linear-5                   [-1, 96]           9,312
              Tanh-6                   [-1, 96]               0
            Linear-7                   [-1, 96]           9,312
              Tanh-8                   [-1, 96]               0
            Linear-9                   [-1, 96]             960
             Tanh-10                   [-1, 96]               0
           Linear-11                   [-1, 96]           9,312
             Tanh-12                   [-1, 96]               0
           Linear-13                   [-1, 96]           9,312
             Tanh-14                   

In [89]:
full_logs_path = os.path.join(os.getcwd() ,log_path)
print("".join(("tensorboard --logdir=",'"',full_logs_path,'"')))

tensorboard --logdir="C:\Users\AlexK\Documents\Python Scripts\RL\logs\SB3_PPO_InvPendulum"


In [91]:
model.learn(total_timesteps=200000, callback=eval_callback, progress_bar=True)

Output()

Eval num_timesteps=10000, episode_reward=110.24 +/- 10.32

Episode length: 13.00 +/- 1.10

New best mean reward!

Eval num_timesteps=20000, episode_reward=234.21 +/- 19.47

...

Episode length: 63.40 +/- 28.67

Eval num_timesteps=140000, episode_reward=7738.46 +/- 2464.96

Episode length: 828.80 +/- 262.99

New best mean reward!

Stopping training because the mean reward 7738.46  is above the threshold 3000

In [100]:
model.save(save_path)

In [93]:
load_path = os.path.join(save_path_best, 'best_model')

model = PPO.load(load_path, env=env)

In [95]:
env = gym.make("InvertedDoublePendulum-v5", render_mode = 'human')
gain, _ = evaluate_policy(model, env, n_eval_episodes=1, render=True)
print(gain)
env.close()

9347.454755783081


In [108]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)

print("Средний доход: ", mean_reward)
print("Стандартное отклонение: ", std_reward)
env.close()

Средний доход:  5789.755002372265
Стандартное отклонение:  2903.251204989814


### Векторизованные среды

Векторизация сред позволяет в фреймворке `SB3` запустить A2C метод, где несколько агентов (Workers) собирают данные. Для этого надо создать векторизованную среду методом `make_vec_env()`.

Сравним работу векторизованной среды и обычной.

In [98]:
import os
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_A2C_CartPole')
log_path = os.path.join('logs', 'SB3_A2C_CartPole')

In [100]:
env = gym.make("CartPole-v1")
model = A2C('MlpPolicy', env, tensorboard_log=log_path)

In [102]:
# векторизованная среда
vec_env = make_vec_env("CartPole-v1", n_envs=4)
model_vec = A2C("MlpPolicy", vec_env, tensorboard_log=log_path)

In [ ]:
full_logs_path = os.path.join(os.getcwd() ,log_path)
print("".join(("tensorboard --logdir=",'"',full_logs_path,'"')))

In [104]:
model.learn(total_timesteps=25000, progress_bar=True)

Output()

In [105]:
model_vec.learn(total_timesteps=25000, progress_bar=True)

Output()

In [108]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)
print(mean_reward)

361.45


In [109]:
mean_reward, std_reward = evaluate_policy(model_vec, env, n_eval_episodes=100, render=False)
print(mean_reward)

500.0


### Ещё несколько примеров сред

In [115]:
import os
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_A2C_InvertedDoublePendulum')
log_path = os.path.join('logs', 'SB3_A2C_InvertedDoublePendulum')

env = gym.make("InvertedDoublePendulum-v5")
model = A2C('MlpPolicy', env, tensorboard_log=log_path)

# векторизованная среда
vec_env = make_vec_env("InvertedDoublePendulum-v5", n_envs=4)
model_vec = A2C("MlpPolicy", vec_env, tensorboard_log=log_path)

In [117]:
full_logs_path = os.path.join(os.getcwd() ,log_path)
print("".join(("tensorboard --logdir=",'"',full_logs_path,'"')))

tensorboard --logdir="C:\Users\AlexK\Documents\Python Scripts\RL\logs\SB3_A2C_InvertedDoublePendulum"


In [119]:
model.learn(total_timesteps=100000, progress_bar=True)

Output()

In [120]:
model_vec.learn(total_timesteps=100000, progress_bar=True)

Output()

In [124]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)
print(mean_reward)

443.04261385083197


In [125]:
mean_reward, std_reward = evaluate_policy(model_vec, env, n_eval_episodes=100, render=False)
print(mean_reward)

481.7737252724171


Пример со средой `MountainCar`, где действия не дискретны, а непрерывны.

In [143]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

save_path = os.path.join('savedModels', 'SB3_PPO_MountainCarContinuous')
log_path = os.path.join('logs', 'SB3_PPO_MountainCarContinuous')

env = gym.make("MountainCarContinuous-v0")
model = PPO('MlpPolicy', env, tensorboard_log=log_path)

# векторизованная среда
vec_env = make_vec_env("MountainCarContinuous-v0", n_envs=4)
model_vec = PPO("MlpPolicy", vec_env, tensorboard_log=log_path)

In [145]:
model.learn(total_timesteps=100000, progress_bar=True)

Output()

In [146]:
model_vec.learn(total_timesteps=100000, progress_bar=True)

Output()

In [147]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, render=False)
print(mean_reward)

-9.670974249588583e-06


In [148]:
mean_reward, std_reward = evaluate_policy(model_vec, env, n_eval_episodes=100, render=False)
print(mean_reward)

-0.001369864273592881


In [153]:
env = gym.make("MountainCarContinuous-v0", render_mode = 'human')
gain, _ = evaluate_policy(model_vec, env, n_eval_episodes=1, render=True)
print(gain)

KeyboardInterrupt: 